# Bayesian Blocks Analysis on a Light Curve

Bayesian Blocks, provide an adaptive, data-driven method for optimal segmentation of time series or event data.
Instead of relying on fixed bin sizes, Bayesian Blocks dynamically determine change points in the data, dividing it into piecewise constant segments (blocks) that best represent the structure of the signal.
We follow the formalization of [Scargle et al. (2013)](https://iopscience.iop.org/article/10.1088/0004-637X/764/2/167).

Key features of the Bayesian Blocks algorithm include:
- Model selection via Bayesian inference: The algorithm selects the most probable segmentation under a Bayesian framework, balancing goodness-of-fit with model complexity.
- Applicability to different data modes: It works with event data (e.g., photon arrival times), regularly sampled time series, and point measurements with errors.
- Non-parametric nature: The method does not assume a particular functional form for the signal, making it robust and flexible.



`Agilepy` implements a `AGBayesianBlocks` class that works with:
- Binned data, i.e. Light Curves. Input data can be provided in 3 different formats: the agile [Aperture Photometry Light Curve](#Bayesian-Blocks-on-Aperture-Photometry-Light-Curves) format, the agile [Maximum Likelihood Light Curve](#Bayesian-Blocks-on-Maximum-Likelihood-Light-Curves) format, a *custom* light curve format.
- [Unbinned](#Bayesian-Blocks-on-Photon-Lists) data, i.e. a list of arrival times of detected photons.

The Bayesian Blocks algorithm has the following steps:
1. Prepare a `YAML` configuration file for `AGBayesianBlocks`. You can do it manually or by using `AGBayesianBlocks.getConfiguration()`.
2. Define a `AGBayesianBlocks` object, and load the data with `AGBayesianBlocks.selectEvents()`.
3. Compute the Bayesian blocks with `AGBayesianBlocks.bayesianBlocks()`.

Note: the custom light curve format is a text file with three columns: bin start time, bin stop times, flux.
Columns do not need an header and must be separated by white spaces.

In [44]:
# Import the relevant classes
import agilepy

from agilepy.api.AGBayesianBlocks import AGBayesianBlocks


## Bayesian Blocks on Aperture Photometry Light Curves

Aperture Photometry Light Curves can be obtained with `AGAnalysis.aperturePhotometry()`.

They are text files with no header and 4 columns separated by white spaces: bin start time, bin stop time, on-source exposure, on-source counts.

In [45]:
# Prepare the Configuration File.
# You can create it manually or use AGBayesianBlocks.getConfiguration

outputDir_ap    = "/home/flareadvocate/workspace/shared_dir/"
confFilePath_ap = "/home/flareadvocate/workspace/shared_dir/tutorial_bb_ap.yaml"

AGBayesianBlocks.getConfiguration(confFilePath=confFilePath_ap,
                                  outputDir=outputDir_ap,
                                  userName="my-name",
                                  sourceName="bb-source",
                                  verboselvl=0,
                                  filePath="$AGILE/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s.ap",
                                  fileMode="AGILE_AP",
                                  tstart="null",
                                  tstop="null",
                                  rateCorrection=0,
                                  fitness="events",
                                  gamma=0.35,
                                  p0="null",
                                  useerror="true"
                                  )

The `YAML` file has the following sections and field:
- `output`: common to all `agilepy` classes, sets the output fields
- `selection`: include arguments used by `AGBayesianBlocks.selectEvents()` used to prepare the data:
    - `filePath`: path to the input file
    - `fileMode`: defines the input file format: `AGILE_AP`, `AGILE_MLE`, `CUSTOM_LC` (binned) or `AGILE_PH` (unbinned).
    - `tstart`, `tstop`: start and stop times for data selection within the input file. For `AGILE_AP` and `AGILE_MLE` must be in `MJD` format, for `CUSTOM_LC`, `AGILE_PH` in the same time format of the times provided.
    - `ratecorrection`: the Bayesian Blocks algorithm works with integers, so fluxes need a correction factor for a correct statistical treatment. If 0 (default), the correction factor is the mean exposure. If a float is provided it will be applied as a correction factor for the flux. If -1, the algorithm will work with counts instead of fluxes.
- `bayesianblocks`: arguments used by `AGBayesianBlocks.bayesianBlocks()` to compute the blocks:
    - `fitness`: set the type of algorithm. Possible values are `events` (binned light curve or unbinned event data), `measures` (sequence of flux measurements with Gaussian errors), `regular_events`(0/1 data measured regularly).
    - `useerror`: boolean to include errors in the computation of the blocks.
    - `gamma`, `p0`: arguments on the prior. `p0`, suggested for unbinned data, gives the false alarm probability to compute the prior. `gamma`, suggested for binned data, gives the slope of the prior on the number of bins.



In [46]:
# Create the AGBayesianBlocks object
ag_bb_ap = AGBayesianBlocks(confFilePath_ap)

Log level set to WARNING and output to /home/flareadvocate/workspace/shared_dir/my-name_bb-source_20250622-124548/logs


In [47]:
# You can Use the setOptions() method to change the configuration, e.g.:
ag_bb_ap.setOptions(gamma=0.35)

# Check configuration.
ag_bb_ap.printOptions()


{ 'bayesianblocks': { 'fitness': 'events',
                      'gamma': 0.35,
                      'p0': None,
                      'useerror': True},
  'output': { 'filenameprefix': 'analysis_product',
              'logfilenameprefix': 'analysis_log',
              'outdir': PosixPath('/home/flareadvocate/workspace/shared_dir/my-name_bb-source_20250622-124548'),
              'sourcename': 'bb-source',
              'username': 'my-name',
              'verboselvl': 0},
  'selection': { 'file_mode': 'AGILE_AP',
                 'file_path': '/home/flareadvocate/agiletools/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s.ap',
                 'ratecorrection': 0,
                 'tstart': None,
                 'tstop': None}}


In [48]:
# Select Events
# If no argument is provided, they are read from the configuration file.
ag_bb_ap.selectEvents()

Select Events. FileMode=AGILE_AP, DataMode=BINNED
Read input file /home/flareadvocate/agiletools/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s.ap
Time Range selected (MJD): 55497.0-55562.0
Total number of rows selected (no zero-exposure): 65
Total number of counts selected (no zero-exposure): 1889
Apply Bayesian Blocks on AP FLUX (= CTS/EXP) scaled to integers with a mean exposure factor=4.54e+06


In [49]:
# You can Inspect the selected data with getDataIn()
selected_events_ap = ag_bb_ap.getDataIn()

In [50]:
# Print
display(selected_events_ap)

# Plot the Data
ag_bb_ap.plotBayesianBlocks()

{'x': array([ 10,  15,  13,  10,  10,  18,  13,  16,  14,  19,  16,  21,  11,
         14,   8,  13,  11,  17,  17,  22,  82, 107, 104, 102,  52,  26,
         43,  32,  30,  36,  44,  43,  40,  35,  33,  30,  29,  25,  18,
         18,  16,  23,  17,  21,  23,  26,  32,  28,  16,  32,  40,  26,
         34,  46,  34,  29,  31,  25,  27,  12,  20,  15,  15,  15,  13]),
 't': array([2.153520e+08, 2.154384e+08, 2.155248e+08, 2.156112e+08,
        2.156976e+08, 2.157840e+08, 2.158704e+08, 2.159568e+08,
        2.160432e+08, 2.161296e+08, 2.162160e+08, 2.163024e+08,
        2.163888e+08, 2.164752e+08, 2.165616e+08, 2.166480e+08,
        2.167344e+08, 2.168208e+08, 2.169072e+08, 2.169936e+08,
        2.170800e+08, 2.171664e+08, 2.172528e+08, 2.173392e+08,
        2.174256e+08, 2.175120e+08, 2.175984e+08, 2.176848e+08,
        2.177712e+08, 2.178576e+08, 2.179440e+08, 2.180304e+08,
        2.181168e+08, 2.182032e+08, 2.182896e+08, 2.183760e+08,
        2.184624e+08, 2.185488e+08, 2.186352e+0

2025-06-22 12:45:48 | WARNING  | agilepy.api.AGBayesianBlocks.AGBayesianBlocks | Bayesian Blocks not computed yet. Plotting only data... | (AGBayesianBlocks.py:255)


In [51]:
# Compute Bayesian Blocks
ag_bb_ap.bayesianBlocks()

/home/flareadvocate/Agilepy/agilepy/external_packages/BBrta/python_code/bayesian_blocks.py:756: RuntimeWarning:

divide by zero encountered in scalar divide



In [52]:
# You can Inspect the results with getDataout()
blocks_computed_ap = ag_bb_ap.getDataOut()

In [53]:
# Print
display(blocks_computed_ap.keys())
display(len(blocks_computed_ap['data_cells']))
display(len(blocks_computed_ap['blockrate']))
display(blocks_computed_ap['N_change_points'])

display(blocks_computed_ap)


dict_keys(['data_cells', 'ncp_prior', 'N', 'change_points', 'edge_points', 'N_data_cells', 'N_change_points', 'edge_vec', 'sum_blocks', 'mean_blocks', 'dt_event_vec', 'dt_block_vec', 'blockrate', 'blockrate2', 'eventrate'])

66

17

16

{'data_cells': array([2.153088e+08, 2.153952e+08, 2.154816e+08, 2.155680e+08,
        2.156544e+08, 2.157408e+08, 2.158272e+08, 2.159136e+08,
        2.160000e+08, 2.160864e+08, 2.161728e+08, 2.162592e+08,
        2.163456e+08, 2.164320e+08, 2.165184e+08, 2.166048e+08,
        2.166912e+08, 2.167776e+08, 2.168640e+08, 2.169504e+08,
        2.170368e+08, 2.171232e+08, 2.172096e+08, 2.172960e+08,
        2.173824e+08, 2.174688e+08, 2.175552e+08, 2.176416e+08,
        2.177280e+08, 2.178144e+08, 2.179008e+08, 2.179872e+08,
        2.180736e+08, 2.181600e+08, 2.182464e+08, 2.183328e+08,
        2.184192e+08, 2.185056e+08, 2.185920e+08, 2.186784e+08,
        2.187648e+08, 2.188512e+08, 2.189376e+08, 2.190240e+08,
        2.191104e+08, 2.191968e+08, 2.192832e+08, 2.193696e+08,
        2.194560e+08, 2.195424e+08, 2.196288e+08, 2.197152e+08,
        2.198016e+08, 2.198880e+08, 2.199744e+08, 2.200608e+08,
        2.201472e+08, 2.202336e+08, 2.203200e+08, 2.204064e+08,
        2.204928e+08, 2.20

In [54]:
# Plot
ag_bb_ap.plotBayesianBlocks(saveImage=False, plotRate=True, plotDataCells=True)

## Bayesian Blocks on Maximum Likelihood Light Curves
Maximum Likelihood Light Curves can be obtained with `AGAnalysis.lightCurve()`.

They are text files with header and multiple columns separated by white spaces.

In [55]:
# Prepare the Configuration File.
# You can create it manually or use AGBayesianBlocks.getConfiguration

outputDir_mle    = "/home/flareadvocate/workspace/shared_dir/"
confFilePath_mle = "/home/flareadvocate/workspace/shared_dir/tutorial_bb_mle.yaml"

AGBayesianBlocks.getConfiguration(confFilePath=confFilePath_mle,
                                  outputDir=outputDir_mle,
                                  userName="my-name",
                                  sourceName="bb-source",
                                  verboselvl=0,
                                  filePath="$AGILE/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s_lc_mle.txt",
                                  fileMode="AGILE_MLE",
                                  tstart="null",
                                  tstop="null",
                                  rateCorrection=0,
                                  fitness="events",
                                  gamma=0.35,
                                  p0="null",
                                  useerror="true"
                                  )

In [56]:
# Create the AGBayesianBlocks object
ag_bb_mle = AGBayesianBlocks(confFilePath_mle)

# You can Use the setOptions() method to change the configuration, e.g.:
# ag_bb_mle.setOptions(gamma=0.35)

# Check configuration.
ag_bb_mle.printOptions()


Log level set to WARNING and output to /home/flareadvocate/workspace/shared_dir/my-name_bb-source_20250622-124552/logs
{ 'bayesianblocks': { 'fitness': 'events',
                      'gamma': 0.35,
                      'p0': None,
                      'useerror': True},
  'output': { 'filenameprefix': 'analysis_product',
              'logfilenameprefix': 'analysis_log',
              'outdir': PosixPath('/home/flareadvocate/workspace/shared_dir/my-name_bb-source_20250622-124552'),
              'sourcename': 'bb-source',
              'username': 'my-name',
              'verboselvl': 0},
  'selection': { 'file_mode': 'AGILE_MLE',
                 'file_path': '/home/flareadvocate/agiletools/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s_lc_mle.txt',
                 'ratecorrection': 0,
                 'tstart': None,
                 'tstop': None}}


In [57]:
# Select Events
ag_bb_mle.selectEvents()
# Inspect
selected_events_mle = ag_bb_mle.getDataIn()

Select Events. FileMode=AGILE_MLE, DataMode=BINNED
Read input file /home/flareadvocate/agiletools/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s_lc_mle.txt
Time Range selected (MJD): 55510.0-55530.0
Total number of rows selected (no zero-exposure, 3sigma): 20
Total number of source photons selected (no zero-exposure, sqrt(ts)>3): 1208.8043
Apply Bayesian Blocks on MLE FLUX scaled to integers with a mean exposure factor=3.54e+06


In [58]:
# Compute Bayesian Blocks
ag_bb_mle.bayesianBlocks()
# Inspect
blocks_computed_mle = ag_bb_mle.getDataOut()

/home/flareadvocate/Agilepy/agilepy/external_packages/BBrta/python_code/bayesian_blocks.py:756: RuntimeWarning:

divide by zero encountered in scalar divide



In [59]:
# Print
display(blocks_computed_mle['N_change_points'])

# Plot
ag_bb_mle.plotBayesianBlocks(saveImage=False, plotRate=True, plotDataCells=True)

8

## Bayesian Blocks on Photon Lists
Photon list files are text files separated by whitespaces where the first column is the photon arrival times.
Please perform any other selection (e.g. position, energy, data quality) beforehand.

In [60]:
# Prepare the Configuration File.
# You can create it manually or use AGBayesianBlocks.getConfiguration

outputDir_ph    = "/home/flareadvocate/workspace/shared_dir/"
confFilePath_ph = "/home/flareadvocate/workspace/shared_dir/tutorial_bb_ph.yaml"
filePath_ph = agilepy.__path__[0]+"/testing/unittesting/api/data/test_data.ph"

AGBayesianBlocks.getConfiguration(confFilePath=confFilePath_ph,
                                  outputDir=outputDir_ph,
                                  userName="my-name",
                                  sourceName="bb-source",
                                  verboselvl=0,
                                  filePath=filePath_ph,
                                  fileMode="AGILE_PH",
                                  tstart="null",
                                  tstop="null",
                                  rateCorrection=0,
                                  fitness="events",
                                  gamma="null",
                                  p0=0.5,
                                  useerror="true"
                                  )


In [61]:
# Create the AGBayesianBlocks object
ag_bb_ph = AGBayesianBlocks(confFilePath_ph)

# Check configuration.
ag_bb_ph.printOptions()

Log level set to WARNING and output to /home/flareadvocate/workspace/shared_dir/my-name_bb-source_20250622-124553/logs
{ 'bayesianblocks': { 'fitness': 'events',
                      'gamma': None,
                      'p0': 0.5,
                      'useerror': True},
  'output': { 'filenameprefix': 'analysis_product',
              'logfilenameprefix': 'analysis_log',
              'outdir': PosixPath('/home/flareadvocate/workspace/shared_dir/my-name_bb-source_20250622-124553'),
              'sourcename': 'bb-source',
              'username': 'my-name',
              'verboselvl': 0},
  'selection': { 'file_mode': 'AGILE_PH',
                 'file_path': '/home/flareadvocate/Agilepy/agilepy/testing/unittesting/api/data/test_data.ph',
                 'ratecorrection': 0,
                 'tstart': None,
                 'tstop': None}}


In [62]:
# Select Events
ag_bb_ph.selectEvents()

# Inspect
selected_events_ph = ag_bb_ph.getDataIn()

Select Events. FileMode=AGILE_PH, DataMode=UNBINNED
Read input file /home/flareadvocate/Agilepy/agilepy/testing/unittesting/api/data/test_data.ph
Number of photons selected: 1285


In [63]:
# Compute Bayesian Blocks
ag_bb_ph.bayesianBlocks()

# Inspect
blocks_computed_ph = ag_bb_ph.getDataOut()

In [64]:
# Print
display(blocks_computed_ph['N_change_points'])

# Plot
ag_bb_ph.plotBayesianBlocks(saveImage=False, plotRate=True)

2